In [1]:
import numpy as np
import matplotlib.pyplot as plt
from heatmap_model.uncertainty_utils import *
%matplotlib inline
import pickle
import seaborn as sns
from scipy.optimize import curve_fit

from skimage.transform import rescale
from skimage import measure


from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

In [2]:
data = np.load('./Uresult/area_e.npz', allow_pickle=True)
A = data['A']/4

with open("./Uresult/polygons.pkl", "rb") as fp:
    b = pickle.load(fp)
P90, _, V, _ = b

B = np.arange(0.2, 4.2, 0.2)

In [4]:
with open('Uresult/polygon_test.pkl', 'wb') as f:
    pickle.dump(P90[:22498], f)

In [ ]:
dz = np.load('./results/testspeed.npz', allow_pickle=True)
Vtest = dz['V']
dz = np.load('./results/valspeed.npz', allow_pickle=True)
Vval = dz['V']

Vr = np.concatenate([Vtest, Vval], 0)

In [ ]:
Vr[:3]

In [ ]:
plt.scatter(V, V/A[13,:,-1], s=0.2)
plt.scatter(V, V/A[-1,:,-1], s=0.2)
plt.show()

In [ ]:
Y = np.zeros((20, 30))
xv = np.linspace(0,28,200)
for i in tqdm(range(30)):
    for j in range(20):
        p = np.polyfit(V,V/A[j,:,i],2)
        Y[j,i] = np.amax(np.polyval(p, xv))

In [ ]:
for i in range(20):
    for j in range(30):
        if j<int(5+i/10):
            Y[i,j] = Y[i,int(5+i/10)]

In [ ]:
Yr = rescale(Y, 2, anti_aliasing=False)*3.25*3600
horizon = np.linspace(0.05, 3.0, 60)
contours = measure.find_contours(Yr, 2000)
buffer = np.linspace(0.2, 4.0, 40)
X_, Y_ = np.meshgrid(horizon, buffer)
fig, ax = plt.subplots(figsize=(7,4))
#rs = ax.contourf(X_, Y_, Yr, zorder=-1)
rs = ax.pcolormesh(X_, Y_, Yr[:40], cmap='rainbow')
CS = ax.contour(X_, Y_, Yr[:40], cmap='inferno_r')
ax.clabel(CS,  inline=False, fontsize=12, colors = 'black')
#ax.hlines(2.0, 0.05, 3.0)
#ax.plot(contours[0][:,1]/60*3+0.05, contours[0][:,0]/10-0.2, c='black', linestyle='--')
plt.xlabel('maximum prediction horizon (s)', fontsize=12)
plt.ylabel('time headway coefficient (s)', fontsize=12)
#plt.text(1.5, 2., '1800', fontsize=12)
cbar = plt.colorbar(rs, label='Road capacity, pcu/(lane$\cdot$h)')
fig.tight_layout()
#plt.yscale('log')
plt.ylim(0.6, 3.6)
#plt.savefig('./imgs/tradeoff.pdf', dpi=600)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(6,4))
ax.plot(contours[0][:,1]/60*3+0.05, contours[0][:,0]/10-0.1, c='black', label='1800 pcu/(lane$\cdot$h)')
plt.xlabel('maximum prediction horizon (s)', fontsize=12)
plt.ylabel('time headway coefficient (s)', fontsize=12)
plt.xlim(0,3.0)
plt.ylim(0.6, 2.5)
ax.hlines(2.0, 0., 3.0, linestyles='--', colors='blue')
ax.hlines(1.5, 0., 3.0, linestyles='--', colors='red')
ax.vlines(1.6, 1.4, 2.4, linestyles='-.', colors='blue')
ax.vlines(2.87, 1.4, 2.4, linestyles='-.', colors='red')

plt.text(1.6, 2.1, '1.61s', fontsize=12)
plt.text(2.5, 1.55, '2.87s', fontsize=12)
plt.legend()
fig.tight_layout()
#plt.savefig('./imgs/tradeoff1800.pdf', dpi=600)
plt.show()

In [ ]:
with open("./Uresult/polygons.pkl", "rb") as fp:
    b = pickle.load(fp)
P90, P80, V, _ = b

In [ ]:
A = []
for d in [i*0.1 for i in range(2, 42, 2)]:
    print(d)
    a = get_occupied_area_test1(P90, V, k=d, safety_buffer=3)
    A.append(a)
A = np.array(A)

In [ ]:
np.savez_compressed('./Uresult/area.npz', A=A, B=np.arange(2, 42, 2)*0.1)

In [ ]:
v = np.linspace(0.1,25,500)
thw = 1.8 + 2/v
flux = v/(1.8*v+2)

fig, ax = plt.subplots(1, 3, figsize=(10.5,3))
ax[0].plot(v, thw, c='blue')
ax[0].hlines(1.8, 0, 30, linestyle='--')
ax[0].set_ylim(0,10)
ax[0].set_xlim(0,15)
ax[0].set_xlabel('speed (m/s)')
ax[0].set_ylabel('time headway (s)')
ax[0].text(2,1,'k')

ax[1].plot(v, flux, c='blue')
ax[1].hlines(1/1.8, 0, 30, linestyle='--')
ax[1].set_ylim(0,0.6)
ax[1].set_xlim(0,25)
ax[1].set_xlabel('speed (m/s)')
ax[1].set_ylabel('flux (pcu/s/m)')
ax[1].text(2,0.5,r'$\frac{1}{2k}$', fontsize=12)

ax[2].plot(xv, yv, c='blue', zorder=10)
ax[2].scatter(V,V/A[6,:,24], color='red', s=0.2, zorder=0, alpha=0.4)
ax[2].set_ylim(0,0.6)
ax[2].set_xlim(0,28)
ax[2].set_xlabel('speed (m/s)')
ax[2].set_ylabel('flux (pcu/s/m)')
#ax[2].text(2,0.25,r'$\frac{1}{2k}$', fontsize=12)

ax[0].set_title('setting of thw coefficient')
ax[1].set_title('ignore prediction uncertainty')
ax[2].set_title('consider prediction uncertainty')

fig.tight_layout()
#plt.savefig('./imgs/fluxexample.pdf', dpi=600)
plt.show()


In [ ]:
Yr.shape

In [ ]:
plt.plot(xv, yv)
plt.scatter(V,V/A[6,:,24])

In [ ]:
a = get_occupied_area_test2(P90, V, k=2.1, safety_buffer=5)

In [ ]:
len(P90[0])